<a href="https://colab.research.google.com/github/Anidafio/4105_hw7/blob/main/hw7_p1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision import datasets, transforms

cifar10 = datasets.CIFAR10(
    root='./data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ]))

cifar10_val = datasets.CIFAR10(
    root='./data', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ]))

cifar10

100%|██████████| 170498071/170498071 [00:03<00:00, 47235679.33it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.view(-1, 3, 32, 32)

        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8) # <1>
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

seq_model = Net()

optimizer = optim.Adam(seq_model.parameters(), lr=1e-3) # <1>

loss_fn = nn.CrossEntropyLoss()
n_epochs = 300
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = seq_model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.003250
Epoch: 1, Loss: 1.807877
Epoch: 2, Loss: 1.645387
Epoch: 3, Loss: 1.494908
Epoch: 4, Loss: 1.367699
Epoch: 5, Loss: 1.244117
Epoch: 6, Loss: 1.155466
Epoch: 7, Loss: 1.101465
Epoch: 8, Loss: 1.062641
Epoch: 9, Loss: 1.031834
Epoch: 10, Loss: 0.996910
Epoch: 11, Loss: 0.963931
Epoch: 12, Loss: 0.932193
Epoch: 13, Loss: 0.900360
Epoch: 14, Loss: 0.867656
Epoch: 15, Loss: 0.841149
Epoch: 16, Loss: 0.820506
Epoch: 17, Loss: 0.805178
Epoch: 18, Loss: 0.792155
Epoch: 19, Loss: 0.778684
Epoch: 20, Loss: 0.765357
Epoch: 21, Loss: 0.755302
Epoch: 22, Loss: 0.748296
Epoch: 23, Loss: 0.739864
Epoch: 24, Loss: 0.731695
Epoch: 25, Loss: 0.726385
Epoch: 26, Loss: 0.719498
Epoch: 27, Loss: 0.716359
Epoch: 28, Loss: 0.709512
Epoch: 29, Loss: 0.702552
Epoch: 30, Loss: 0.694284
Epoch: 31, Loss: 0.684283
Epoch: 32, Loss: 0.674217
Epoch: 33, Loss: 0.664714
Epoch: 34, Loss: 0.653464
Epoch: 35, Loss: 0.637832
Epoch: 36, Loss: 0.624889
Epoch: 37, Loss: 0.610369
Epoch: 38, Loss: 0.598

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = seq_model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.726220


In [11]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = seq_model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.568300
